In [1]:
import pandas as pd

In [2]:
actor_movies_revenue_filtered = pd.read_csv("actor_movies_revenue_filtered.csv")
actor_movies_revenue_filtered.head(3)

,tconst,Money_Type,Money_Amount
0,tt0035423,Domestic (62%),"$47,121,859"
1,tt0120903,Domestic (53.1%),"$157,299,718"
2,tt0180037,Missing,-999


In [3]:
actor_movies_revenue_dropmissing = actor_movies_revenue_filtered[actor_movies_revenue_filtered['Money_Type']!='Missing'].copy()
actor_movies_revenue_dropmissing.head(3)

,tconst,Money_Type,Money_Amount
0,tt0035423,Domestic (62%),"$47,121,859"
1,tt0120903,Domestic (53.1%),"$157,299,718"
4,tt0244244,Domestic (47.4%),"$69,772,969"


In [6]:
actor_movies_revenue_dropmissing['Money_Amount_Num'] = actor_movies_revenue_dropmissing['Money_Amount'].replace('[\$,]', '', regex=True).astype(int)

In [12]:
actor_movies_ratings = pd.read_csv("actor_movies_ratings.csv", na_values='\\N').drop_duplicates(subset=['tconst','nconst'])
print(actor_movies_ratings.shape)
actor_movies_ratings.head(3)

/var/folders/7v/x80m047d1mx4nq6f2whpjndw0000gn/T/ipykernel_3866/2911724416.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  actor_movies_ratings = pd.read_csv("actor_movies_ratings.csv", na_values='\\N').drop_duplicates(subset=['tconst','nconst'])


(313009, 13)


,tconst,ordering,nconst,category,job,characters,averageRating,numVotes,primaryTitle,startYear,region,primaryName,birthYear
0,tt0035423,2,nm0413168,actor,NaN,"[""Leopold""]",6.4,88860,Kate & Leopold,2001.0,US,Hugh Jackman,1968.0
2,tt0120903,2,nm0413168,actor,NaN,"[""Logan""]",7.3,644121,X-Men,2000.0,US,Hugh Jackman,1968.0
8,tt0180037,2,nm0413168,actor,NaN,"[""Jack Willis""]",6.4,2344,Paperback Hero,1999.0,US,Hugh Jackman,1968.0


In [15]:
actor_movies_ratings_dropna = actor_movies_ratings[~actor_movies_ratings['birthYear'].isna()].copy()

In [18]:
actor_movies_ratings_revenue = actor_movies_ratings_dropna.merge(actor_movies_revenue_dropmissing, how='inner', on='tconst')
actor_movies_ratings_revenue.head(3)

,tconst,ordering,nconst,category,job,characters,averageRating,numVotes,primaryTitle,startYear,region,primaryName,birthYear,age_at_movie,Money_Type,Money_Amount,Money_Amount_Num
0,tt0035423,2,nm0413168,actor,NaN,"[""Leopold""]",6.4,88860,Kate & Leopold,2001,US,Hugh Jackman,1968,33,Domestic (62%),"$47,121,859",47121859
1,tt0035423,3,nm0000630,actor,NaN,"[""Stuart Besser""]",6.4,88860,Kate & Leopold,2001,US,Liev Schreiber,1967,34,Domestic (62%),"$47,121,859",47121859
2,tt0035423,4,nm0005227,actor,NaN,"[""Charlie McKay""]",6.4,88860,Kate & Leopold,2001,US,Breckin Meyer,1974,27,Domestic (62%),"$47,121,859",47121859


In [19]:
all_years = actor_movies_ratings_revenue[['startYear']].drop_duplicates().sort_values('startYear')
all_years.head(3)

,startYear
570,1980
1093,1981
576,1982


In [36]:
all_actors = actor_movies_ratings_revenue[['nconst','primaryName','birthYear']].drop_duplicates()
all_actors.head(3)

,nconst,primaryName,birthYear
0,nm0413168,Hugh Jackman,1968
1,nm0000630,Liev Schreiber,1967
2,nm0005227,Breckin Meyer,1974


In [37]:
all_years['key'] = 0
all_actors['key'] = 0
all_years_all_actors = all_years.merge(all_actors, on='key', how='outer')
all_years_all_actors.sort_values(['nconst','startYear']).head(3)

,startYear,key,nconst,primaryName,birthYear
3404,1980,0,nm0000008,Marlon Brando,1924
10830,1981,0,nm0000008,Marlon Brando,1924
18256,1982,0,nm0000008,Marlon Brando,1924


In [41]:
actors_with_all_years_revenue = all_years_all_actors.merge(actor_movies_ratings_revenue[['tconst', 'nconst', 'averageRating', 'numVotes', 'primaryTitle', 'startYear'
                                                                                         , 'Money_Type', 'Money_Amount_Num']]
                           , how='left', on=['startYear','nconst']).drop(labels='key', axis=1)

actors_with_all_years_revenue.head(3)

,startYear,nconst,primaryName,birthYear,tconst,averageRating,numVotes,primaryTitle,Money_Type,Money_Amount_Num
0,1980,nm0413168,Hugh Jackman,1968,NaN,NaN,NaN,NaN,NaN,NaN
1,1980,nm0000630,Liev Schreiber,1967,NaN,NaN,NaN,NaN,NaN,NaN
2,1980,nm0005227,Breckin Meyer,1974,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
actors_with_all_years_revenue_grouped = actors_with_all_years_revenue.groupby(['startYear','nconst','primaryName','birthYear']).agg(sumaverageRating=('averageRating','sum')
                                                                                            ,sumnumVotes=('numVotes','sum')
                                                                                            ,Money_Amount_Num=('Money_Amount_Num','sum')
                                                                                            ,primaryTitle=('primaryTitle',list)).reset_index()

In [81]:
actors_with_all_years_revenue_grouped.head(3)

,startYear,nconst,primaryName,birthYear,sumaverageRating,sumnumVotes,Money_Amount_Num,primaryTitle,has_movie,age_at_movie,rolling_5yr_avgrevenue,rolling_5yr_sumrevenue,max_rolling_5yr_sumrevenue
0,1980,nm0000008,Marlon Brando,1924,0.0,0.0,0.0,[nan],True,56,0.0,0.0,71107711.0
1,1980,nm0000018,Kirk Douglas,1916,0.0,0.0,0.0,[nan],True,64,0.0,0.0,9261.0
2,1980,nm0000024,John Gielgud,1904,8.2,257411.0,26010864.0,[The Elephant Man],True,76,26010864.0,26010864.0,78778753.0


In [57]:
actors_with_all_years_revenue_grouped['has_movie'] = ~actors_with_all_years_revenue_grouped['primaryTitle'].isna()
actors_with_all_years_revenue_grouped['startYear'] = actors_with_all_years_revenue_grouped['startYear'].astype(int)
actors_with_all_years_revenue_grouped['birthYear'] = actors_with_all_years_revenue_grouped['birthYear'].astype(int)
actors_with_all_years_revenue_grouped['age_at_movie'] = actors_with_all_years_revenue_grouped['startYear']-actors_with_all_years_revenue_grouped['birthYear']

In [66]:
# actors_with_all_years_revenue['rolling_5yr_movies'] = actors_with_all_years_revenue.groupby('primaryName')['has_movie'].transform(lambda s: s.rolling(5, min_periods=1).sum())

# actors_with_all_years_revenue['rolling_5yr_avgrating'] = actors_with_all_years_revenue.groupby('primaryName')['averageRating'].transform(lambda s: s.rolling(5, min_periods=1).mean())
# actors_with_all_years_revenue['rolling_5yr_sumrating'] = actors_with_all_years_revenue.groupby('primaryName')['averageRating'].transform(lambda s: s.rolling(5, min_periods=1).sum())
# actors_with_all_years_revenue['max_rolling_5yr_sumrating'] = actors_with_all_years_revenue.groupby('primaryName')['rolling_5yr_sumrating'].transform('max')

actors_with_all_years_revenue_grouped['rolling_5yr_avgrevenue'] = actors_with_all_years_revenue_grouped.groupby('primaryName')['Money_Amount_Num'].transform(lambda s: s.rolling(5, min_periods=1).mean())
actors_with_all_years_revenue_grouped['rolling_5yr_sumrevenue'] = actors_with_all_years_revenue_grouped.groupby('primaryName')['Money_Amount_Num'].transform(lambda s: s.rolling(5, min_periods=1).sum())
actors_with_all_years_revenue_grouped['max_rolling_5yr_sumrevenue'] = actors_with_all_years_revenue_grouped.groupby('primaryName')['rolling_5yr_sumrevenue'].transform('max')

In [82]:
actors_with_all_years_revenue_grouped['primaryTitle_str'] = actors_with_all_years_revenue_grouped['primaryTitle'].apply(lambda x: ",".join([item for item in x if item==item]))

In [85]:
image_link_list_w_name_all = pd.read_csv("image_link_list_w_name_all.csv", usecols=['nconst','wiki_image_link'])
image_link_list_w_name_all.head(3)

,nconst,wiki_image_link
0,nm0000685,https://upload.wikimedia.org/wikipedia/commons...
1,nm1221047,https://upload.wikimedia.org/wikipedia/commons...
2,nm0004335,https://upload.wikimedia.org/wikipedia/commons...


In [86]:
actors_with_all_years_revenue_links = actors_with_all_years_revenue_grouped.merge(image_link_list_w_name_all, how='inner', on='nconst').drop('primaryTitle', axis=1)
actors_with_all_years_revenue_links.head(3)

,startYear,nconst,primaryName,birthYear,sumaverageRating,sumnumVotes,Money_Amount_Num,has_movie,age_at_movie,rolling_5yr_avgrevenue,rolling_5yr_sumrevenue,max_rolling_5yr_sumrevenue,primaryTitle_str,wiki_image_link
0,1980,nm0000092,John Cleese,1939,0.0,0.0,0.0,True,41,0.000000e+00,0.0,287885823.0,,https://upload.wikimedia.org/wikipedia/commons...
1,1981,nm0000092,John Cleese,1939,6.9,68046.0,42365581.0,True,42,2.118279e+07,42365581.0,287885823.0,Time Bandits,https://upload.wikimedia.org/wikipedia/commons...
2,1982,nm0000092,John Cleese,1939,0.0,0.0,0.0,True,43,1.412186e+07,42365581.0,287885823.0,,https://upload.wikimedia.org/wikipedia/commons...


In [88]:
actors_with_all_years_revenue_links.columns

Index(['startYear', 'nconst', 'primaryName', 'birthYear', 'sumaverageRating',
       'sumnumVotes', 'Money_Amount_Num', 'has_movie', 'age_at_movie',
       'rolling_5yr_avgrevenue', 'rolling_5yr_sumrevenue',
       'max_rolling_5yr_sumrevenue', 'primaryTitle_str', 'wiki_image_link'],
      dtype='object')

In [90]:
actors_with_all_years_revenue_links_for_export = actors_with_all_years_revenue_links[['startYear', 'primaryName', 'birthYear'
                                                                                        , 'age_at_movie','rolling_5yr_sumrevenue',
                                                                                    'max_rolling_5yr_sumrevenue', 'primaryTitle_str', 'wiki_image_link']].copy()

In [91]:
actors_with_all_years_revenue_links_for_export.to_csv("actors_with_all_years_revenue_links.csv", index=False)

In [97]:
actors_with_all_years_revenue_links_for_export[actors_with_all_years_revenue_links_for_export['max_rolling_5yr_sumrevenue']>=7.0e+8].to_csv("actors_with_all_years_revenue_links_filtered.csv", index=False)

In [93]:
9.0e+8

900000000.0